In [28]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import seaborn    as sns
from sklearn               import model_selection as ms
from sklearn               import ensemble        as en
from sklearn               import preprocessing as pp
from sklearn.metrics       import mean_absolute_error,mean_squared_error
from sklearn.linear_model  import LinearRegression,Lasso

In [46]:
def mean_percentage_error(y_real,y_estimado):
    """Definindo a função do MPE.
    
    y_real - dados reais do fenômeno
    y_estimado - dados estimados pelo modelo - yhat
    """
    return np.mean((y_real - y_estimado)/y_real)

def mean_absolute_percentage_error(y_real,y_estimado):
    """Função para calcular o MAE, MAPE e RMSE de cada modelo aplicado.
    
    y_real - dados reais do fenômeno
    y_estimado - dados estimados pelo modelo - yhat
    """
    return np.mean(np.abs((y_real - y_estimado)/y_real))

def ml_error(model_name,y_real,y_estimado):
    """Função para calcular o MAE, MAPE e RMSE de cada modelo aplicado.
    
    model_name - String do nome do modelo
    y_real - dados reais do fenômeno
    y_estimado - dados estimados pelo modelo - yhat
    """
    mae = mean_absolute_error(y_real,y_estimado)
    mape = mean_absolute_percentage_error(y_real,y_estimado)
    rmse = np.sqrt(mean_squared_error(y_real,y_estimado))

    return pd.DataFrame({'Model Name':model_name,
                        'MAE':mae,
                             'MAPE':mape,
                             'RMSE':rmse},index=[0])

In [85]:
def cross_validation(model,x_train,y_train,k,Verbose=True):
    """Função para treinar o modelo em diferentes espaços das amotra e retornar o valor dos eus erros, juntamente com seus ranges.
    
    x_training - Dataset de Treino
    kfold - Número de intervalos em que o modelo será treinado
    model_name - String do nome do modelo
    model - Modelo já instanciado
    verbose - Vizualização do progresso do Cross Validation
    """
    #Selecionar o primeiro espaço de validação, 6 semanas até o fim do treino - start and end for date validation
    kfold = ms.KFold(n_splits=k, shuffle=True, random_state=42)
    mae_list = []
    mape_list = []
    rmse_list = []
    i = 1
    for train_cv, val_cv in kfold.split(x_train, y_train):
        if Verbose == True:    
            print(f'Fold Number {i}/{k}') 
        else:
            pass
        
        x_train_fold = x_train.iloc[train_cv]
        y_train_fold = y_train.iloc[train_cv]
        
        x_val_fold = x_train.iloc[val_cv]
        y_val_fold = y_train.iloc[val_cv]

        # model
        model.fit(x_train_fold, y_train_fold)
        
        yhat_model = model.predict(x_val_fold)

        # performance - Y_test é a coluna de sales que devemos encontrar e yhat é o que nosso método prediz
        m_result = ml_error(type(model).__name__, y_val_fold, yhat_model)

        # Guardando a performance de cada iteração
        mae_list.append(m_result['MAE'])
        mape_list.append(m_result['MAPE'])
        rmse_list.append(m_result['RMSE'])
        i += 1

    return pd.DataFrame({'Model Name': type(model).__name__,
                     'MAE CV': np.round(np.mean(mae_list), 2).astype(str) + ' +/- ' + np.round(np.std(mae_list), 2).astype(str),
                     'MAPE CV': np.round(np.mean(mape_list), 2).astype(str) + ' +/- ' + np.round(np.std(mape_list), 2).astype(str),
                     'RSME CV': np.round(np.mean(rmse_list), 2).astype(str) + ' +/- ' + np.round(np.std(rmse_list), 2).astype(str)}, index=[0])

In [13]:
cols = ['idcsdmpessoa','Receita','Inatividade','Compras','mesesCadastro','vlrCLV']
clientes_outubro = pd.read_csv("clientes_outubro.csv", names=cols,sep=';',low_memory=False)

df1 = clientes_outubro.copy()

In [14]:
df1['vlrCLV'] = df1['vlrCLV'].str.replace(',', '.').astype(float)
df1.head()

,idcsdmpessoa,Receita,Inatividade,Compras,mesesCadastro,vlrCLV
0,4506,387.9420,20,7,63,905.482003
1,5521,4666.7395,20,25,63,10976.409685
2,422022,4630.0577,17,32,62,12074.046757
3,423314,2891.8703,65,22,252,7874.528815
4,423812,764.3719,6,12,213,2007.965663


In [76]:
#Simulando os dados que vem em produção
X = df1.drop('vlrCLV',axis=1)

#Guardando a variável resposta
Y = df1['vlrCLV'].copy()

#Separando de forma aleatória os dados que vão para o treino e os que vão para o teste
x_train, x_validation, y_train, y_validation = ms.train_test_split(X,Y,test_size=0.20)

#Juntando os dados de treino com a variável resposta do treino
#df5 = pd.concat([x_train,y_train],axis=1)

In [77]:
mm = pp.MinMaxScaler() #Mantem a mesma distribuição, com outro shape, e colocando de 0 a 1
ss = pp.StandardScaler() #Rescala os dados

x_train['Receita'] = mm.fit_transform(x_train[['Receita']]) 
x_train['Inatividade'] = mm.fit_transform(x_train[['Inatividade']])
x_train['Compras'] = mm.fit_transform(x_train[['Compras']]) 
x_train['mesesCadastro'] = mm.fit_transform(x_train[['mesesCadastro']])
#y_train = np.log(y_train)

x_validation['Receita'] = mm.fit_transform(x_validation[['Receita']]) 
x_validation['Inatividade'] = mm.fit_transform(x_validation[['Inatividade']])
x_validation['Compras'] = mm.fit_transform(x_validation[['Compras']]) 
x_validation['mesesCadastro'] = mm.fit_transform(x_validation[['mesesCadastro']])
#y_validation = np.log(y_validation)

In [87]:
#model
lr = LinearRegression().fit(x_train,y_train)

#prediction
yhat_lr = lr.predict(x_validation)

#performance - Y_test é a coluna de sales que devemos encontrar e yhat_lr é o que nosso método prediz
lr_result = ml_error('Linear Regression',y_validation,yhat_lr)
lr_result

,Model Name,MAE,MAPE,RMSE
0,Linear Regression,26111.888828,15.269128,55494.046169


In [88]:
lr_result_cv = cross_validation(lr,x_train,y_train,5,Verbose=True)
lr_result_cv

Fold Number 1/5
Fold Number 2/5
Fold Number 3/5
Fold Number 4/5
Fold Number 5/5


,Model Name,MAE CV,MAPE CV,RSME CV
0,LinearRegression,565.38 +/- 14.76,8.26 +/- 0.34,1029.15 +/- 180.05


In [91]:
#model
lrr = Lasso(alpha=0.01).fit(x_train,y_train)

#prediction
yhat_lrr = lrr.predict(x_validation)

#performance - Y_test é a coluna de sales que devemos encontrar e yhat_lr é o que nosso método prediz
lr_result = ml_error('Linear Regression Lasso',y_validation,yhat_lrr)
lr_result

,Model Name,MAE,MAPE,RMSE
0,Linear Regression Lasso,26089.939152,15.265353,55446.119182


In [90]:
lrr_result_cv = cross_validation(lrr,x_train,y_train,5,Verbose=True)
lrr_result_cv

Fold Number 1/5
Fold Number 2/5
Fold Number 3/5
Fold Number 4/5
Fold Number 5/5


,Model Name,MAE CV,MAPE CV,RSME CV
0,Lasso,564.3 +/- 14.63,8.27 +/- 0.34,1029.99 +/- 184.31


In [92]:
#model
rf = en.RandomForestRegressor(n_estimators=100, n_jobs=-1,random_state=42).fit(x_train,y_train)

#prediction
yhat_rf = rf.predict(x_validation)

#performance - Y_test é a coluna de sales que devemos encontrar e yhat_lr é o que nosso método prediz
lr_result = ml_error('Random Forest',y_validation,yhat_rf)
lr_result